In [111]:
import pandas as pd
import sklearn as skl
import xgboost as xgb
import category_encoders as ce
from datetime import timedelta
from sklearn.metrics import explained_variance_score


cols = ['created','ref_hash','implicit','device_brand','device_model','wifi','device_language','ref_type']
installs = pd.read_csv('/home/fedoremelianenko/FIUBA/Organizacion de datos/TPs/TP2/data/installs.csv',usecols=cols)
installs['created'] = pd.to_datetime(installs['created'])

def generar_ventanas(df, str_columna_fecha):
    FECHA_INICIO = pd.to_datetime(df[str_columna_fecha].min().strftime('%Y-%m-%d'))
    FECHA_FIN = pd.to_datetime(df[str_columna_fecha].max().strftime('%Y-%m-%d'))

    FECHA_UN_DIA = FECHA_INICIO + timedelta(days=1)
    FECHA_DOS_DIAS = FECHA_INICIO + timedelta(days=2)
    FECHA_TRES_DIAS = FECHA_INICIO + timedelta(days=3)
    FECHA_CUATRO_DIAS = FECHA_INICIO + timedelta(days=4)
    FECHA_CINCO_DIAS = FECHA_INICIO + timedelta(days=5)
    FECHA_SEIS_DIAS = FECHA_INICIO + timedelta(days=6)
    FECHA_SIETE_DIAS = FECHA_INICIO + timedelta(days=7)

    ventana_uno = installs.loc[(df[str_columna_fecha] < FECHA_TRES_DIAS)]
    ventana_uno = ventana_uno.loc[(df[str_columna_fecha] > FECHA_INICIO)]

    ventana_dos = installs.loc[(df[str_columna_fecha] < FECHA_CUATRO_DIAS)]
    ventana_dos = ventana_dos.loc[(df[str_columna_fecha] > FECHA_UN_DIA)]

    ventana_tres = installs.loc[(df[str_columna_fecha] < FECHA_CINCO_DIAS)]
    ventana_tres = ventana_tres.loc[(df[str_columna_fecha] > FECHA_DOS_DIAS)]
    
    ventana_cuatro = installs.loc[(df[str_columna_fecha] < FECHA_SEIS_DIAS)]
    ventana_cuatro = ventana_tres.loc[(df[str_columna_fecha] > FECHA_TRES_DIAS)]
    
    ventana_cinco = installs.loc[(df[str_columna_fecha] < FECHA_SIETE_DIAS)]
    ventana_cinco = ventana_tres.loc[(df[str_columna_fecha] > FECHA_CUATRO_DIAS)]
    
    return ventana_uno, ventana_dos, ventana_tres, ventana_cuatro, ventana_cinco

In [112]:
installs = installs.loc[installs['implicit'] == True]
installs['wifi'] = installs['wifi'].apply(lambda x: 1 if x== True else 0)
installs['ref_type'] = installs['ref_type'].apply(lambda x: 1 if (x == 1891515180541284343) else 0)
installs['device_language']=installs['device_language'].fillna(value=0)
installs.head()

,created,ref_type,ref_hash,implicit,device_brand,device_model,wifi,device_language
0,2019-04-24 06:23:29.495,0,4716708407362582887,True,NaN,3.739127e+17,0,3.301378e+18
2,2019-04-20 10:15:36.274,0,5230323462636548010,True,NaN,8.355496e+18,0,4.060930e+18
3,2019-04-20 21:56:47.151,0,5097163995161606833,True,NaN,2.355772e+18,0,3.301378e+18
5,2019-04-20 18:19:27.485,0,7522785771858684314,True,NaN,6.208879e+18,0,3.301378e+18
6,2019-04-20 03:40:21.239,0,7882044913917355073,True,NaN,4.566898e+18,0,4.060930e+18


In [113]:
installs['cantidad_instalaciones'] = 1

In [114]:
installs_ventana_uno, installs_ventana_dos, installs_ventana_tres, installs_ventana_cuatro, installs_ventana_cinco = generar_ventanas(installs,'created')

In [115]:
iv1_grouped = installs_ventana_uno.groupby('ref_hash').agg({'cantidad_instalaciones':'sum'})
installs_ventana_uno = installs_ventana_uno.merge(iv1_grouped, on='ref_hash',how='left')
installs_ventana_uno.drop(['cantidad_instalaciones_x'], axis=1, inplace=True)
installs_ventana_uno.rename({'cantidad_instalaciones_y':'cantidad_instalaciones'}, inplace=True, axis=1)

iv2_grouped = installs_ventana_dos.groupby('ref_hash').agg({'cantidad_instalaciones':'sum'})
installs_ventana_dos = installs_ventana_dos.merge(iv2_grouped, on='ref_hash',how='left')
installs_ventana_dos.drop(['cantidad_instalaciones_x'], axis=1, inplace=True)
installs_ventana_dos.rename({'cantidad_instalaciones_y':'cantidad_instalaciones'}, inplace=True, axis=1)

iv3_grouped = installs_ventana_tres.groupby('ref_hash').agg({'cantidad_instalaciones':'sum'})
installs_ventana_tres = installs_ventana_tres.merge(iv3_grouped, on='ref_hash',how='left')
installs_ventana_tres.drop(['cantidad_instalaciones_x'], axis=1, inplace=True)
installs_ventana_tres.rename({'cantidad_instalaciones_y':'cantidad_instalaciones'}, inplace=True, axis=1)

iv4_grouped = installs_ventana_cuatro.groupby('ref_hash').agg({'cantidad_instalaciones':'sum'})
installs_ventana_cuatro = installs_ventana_uno.merge(iv4_grouped, on='ref_hash',how='left')
installs_ventana_cuatro.drop(['cantidad_instalaciones_x'], axis=1, inplace=True)
installs_ventana_cuatro.rename({'cantidad_instalaciones_y':'cantidad_instalaciones'}, inplace=True, axis=1)

iv5_grouped = installs_ventana_cinco.groupby('ref_hash').agg({'cantidad_instalaciones':'sum'})
installs_ventana_cinco = installs_ventana_cinco.merge(iv5_grouped, on='ref_hash',how='left')
installs_ventana_cinco.drop(['cantidad_instalaciones_x'], axis=1, inplace=True)
installs_ventana_cinco.rename({'cantidad_instalaciones_y':'cantidad_instalaciones'}, inplace=True, axis=1)

In [116]:
def t_sum(t_inst):
    suma = (t_inst.iloc[0] - pd.to_datetime(pd.to_datetime(t_inst.iloc[0]).strftime('%Y-%m-%d'))).total_seconds()
    for i in range(1, len(t_inst)):
        suma += (t_inst.iloc[i] - t_inst.iloc[i-1]).total_seconds()
    return suma

def t_hasta_conversion(t_inst):
    return (t_inst.iloc[0] - pd.to_datetime(pd.to_datetime(t_inst.iloc[0]).strftime('%Y-%m-%d'))).total_seconds()

installs_ventana_uno.sort_values(by='created',axis=0,inplace=True)
installs_ventana_uno['primera_instalacion'] = installs_ventana_uno['created']

installs_ventana_dos.sort_values(by='created',axis=0,inplace=True)
installs_ventana_dos['primera_instalacion'] = installs_ventana_dos['created']

installs_ventana_tres.sort_values(by='created',axis=0,inplace=True)
installs_ventana_tres['primera_instalacion'] = installs_ventana_tres['created']

installs_ventana_cuatro.sort_values(by='created',axis=0,inplace=True)
installs_ventana_cuatro['primera_instalacion'] = installs_ventana_cuatro['created']

installs_ventana_cinco.sort_values(by='created',axis=0,inplace=True)
installs_ventana_cinco['primera_instalacion'] = installs_ventana_cinco['created']

iv1 = installs_ventana_uno.groupby(['ref_hash','ref_type','device_brand','device_model','device_language',
                                    'cantidad_instalaciones']).agg({'created':t_sum,'wifi':'mean',
                                                                   'primera_instalacion':t_hasta_conversion})

iv2 = installs_ventana_dos.groupby(['ref_hash','ref_type','device_brand','device_model','device_language',
                                    'cantidad_instalaciones']).agg({'created':t_sum,'wifi':'mean',
                                                                   'primera_instalacion':t_hasta_conversion})

iv3 = installs_ventana_tres.groupby(['ref_hash','ref_type','device_brand','device_model','device_language',
                                    'cantidad_instalaciones']).agg({'created':t_sum,'wifi':'mean',
                                                                   'primera_instalacion':t_hasta_conversion})

iv4 = installs_ventana_cuatro.groupby(['ref_hash','ref_type','device_brand','device_model','device_language',
                                    'cantidad_instalaciones']).agg({'created':t_sum,'wifi':'mean',
                                                                   'primera_instalacion':t_hasta_conversion})

iv5 = installs_ventana_cinco.groupby(['ref_hash','ref_type','device_brand','device_model','device_language',
                                    'cantidad_instalaciones']).agg({'created':t_sum,'wifi':'mean',
                                                                   'primera_instalacion':t_hasta_conversion})
iv1.reset_index(inplace=True)
iv1.rename({'created':'t_sum'},inplace=True, axis=1)

iv2.reset_index(inplace=True)
iv2.rename({'created':'t_sum'},inplace=True, axis=1)

iv3.reset_index(inplace=True)
iv3.rename({'created':'t_sum'},inplace=True, axis=1)

iv4.reset_index(inplace=True)
iv4.rename({'created':'t_sum'},inplace=True, axis=1)

iv5.reset_index(inplace=True)
iv5.rename({'created':'t_sum'},inplace=True, axis=1)

In [117]:
'''encoder_bi = ce.BinaryEncoder(cols=['device_model','device_language','device_brand'])
iv1 = encoder_bi.fit_transform(iv1)
iv2 = encoder_bi.fit_transform(iv2)
iv3 = encoder_bi.fit_transform(iv3)
iv4 = encoder_bi.fit_transform(iv4)
iv5 = encoder_bi.fit_transform(iv5)'''

"encoder_bi = ce.BinaryEncoder(cols=['device_model','device_language','device_brand'])\niv1 = encoder_bi.fit_transform(iv1)\niv2 = encoder_bi.fit_transform(iv2)\niv3 = encoder_bi.fit_transform(iv3)\niv4 = encoder_bi.fit_transform(iv4)\niv5 = encoder_bi.fit_transform(iv5)"

In [118]:
ventanaUnoData = iv1.drop(['primera_instalacion'], axis=1).values
ventanaUnoTargets = iv1['primera_instalacion'].values

ventanaDosData = iv2.drop(['primera_instalacion'],axis=1).values
ventanaDosTargets = iv2['primera_instalacion'].values

ventanaTresData = iv3.drop(['primera_instalacion'], axis=1).values
ventanaTresTargets = iv3['primera_instalacion'].values

ventanaCuatroData = iv4.drop(['primera_instalacion'], axis=1).values
ventanaCuatroTargets = iv4['primera_instalacion'].values

ventanaCincoData = iv5.drop(['primera_instalacion'], axis=1).values
ventanaCincoTargets = iv5['primera_instalacion'].values


xgbVentanaUno = xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7, objective='reg:squarederror')

xgbVentanaDos = xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7, objective='reg:squarederror')

xgbVentanaTres = xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7, objective='reg:squarederror')

xgbVentanaCuatro = xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7, objective='reg:squarederror')

xgbVentanaCinco = xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7, objective='reg:squarederror')

In [119]:
xgbVentanaUno.fit(ventanaUnoData, ventanaUnoTargets)
xgbVentanaDos.fit(ventanaDosData, ventanaDosTargets)
xgbVentanaTres.fit(ventanaTresData, ventanaTresTargets)
xgbVentanaCuatro.fit(ventanaCuatroData, ventanaCuatroTargets)
xgbVentanaCinco.fit(ventanaCincoData, ventanaCincoTargets)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.08, max_delta_step=0,
             max_depth=7, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.75, verbosity=1)

In [132]:
prediccionV1_V2 = xgbVentanaUno.predict(ventanaDosData)
prediccionV2_V3 = xgbVentanaDos.predict(ventanaTresData)
prediccionV3_V4 = xgbVentanaTres.predict(ventanaCuatroData)
prediccionV4_V5 = xgbVentanaCuatro.predict(ventanaCincoData)
prediccionV5_V1 = xgbVentanaCinco.predict(ventanaUnoData)

prediccionV1_V4 = xgbVentanaUno.predict(ventanaCuatroData)
prediccionV2_V5 = xgbVentanaDos.predict(ventanaCincoData)

In [125]:
explained_variance_score(prediccionV1_V2, ventanaDosTargets)

0.9950602815853041

In [126]:
explained_variance_score(prediccionV2_V3, ventanaTresTargets)

0.994862301284657

In [127]:
explained_variance_score(prediccionV3_V4, ventanaCuatroTargets)

0.9659257467602657

In [128]:
explained_variance_score(prediccionV4_V5, ventanaCincoTargets)

0.9880996548486832

In [130]:
explained_variance_score(prediccionV1_V4, ventanaCuatroTargets)

0.9793026402661242

In [133]:
explained_variance_score(prediccionV2_V5, ventanaCincoTargets)

0.9959225844731532